In [ ]:
import geopandas as gpd
import pandas as pd
import pathlib

import matplotlib.pyplot as plt

from cartopy import crs as ccrs

import os
import sys
import subprocess
import datetime
import platform
import datetime
from folium.plugins import StripePattern

In [ ]:
import glob
path=r'census_shp'

file=glob.glob(os.path.join(path, 'cb_2018_**_tract_500k.shp'))
# print(file)
print(len(file))

In [ ]:
basemap= gpd.GeoDataFrame()
for f in file:
    basemap = pd.concat([basemap, gpd.read_file(f)], axis=0)

In [ ]:
basemap = basemap[['GEOID','AFFGEOID','STATEFP','geometry']]
basemap.head()

In [ ]:
basemap.crs

In [ ]:
basemap=basemap.to_crs("ESRI:102003")

In [ ]:
basemap["geometry"] = basemap["geometry"].simplify(5000)

In [ ]:
basemap=basemap.to_crs("epsg:4326")

In [ ]:
basemap.crs

In [ ]:
us_jason = basemap.to_json()

In [ ]:
us_jason

# Folium

In [ ]:
exp = pd.read_csv('census_data/exposure_census.csv', dtype = {'State':str,"GEOID":str})
exp = exp.rename(columns = {'min-max':'Exposure Score'})
exp = exp[['GEOID', 'Exposure Score']]
exp.head()

In [ ]:
sens = pd.read_csv("census_data/sensitivity_census.csv", dtype={"Geocode": str})
sens = sens.rename(columns = {'min_max_sens':'Sensitivity Score', 'Geocode':'GEOID'})
sens = sens[['GEOID', 'Sensitivity Score']]
sens.head()

In [ ]:
ac = pd.read_csv('census_data/adaptive_capacity_census.csv', dtype={"GEO_ID":str})
ac = ac.rename(columns = {'AC_score':'Adaptive Capacity Score', 'GEO_ID':'GEOID'})
ac = ac[['GEOID', 'Adaptive Capacity Score']]
ac.head()

In [ ]:
vul = pd.read_csv('Vulnerability Score.csv', dtype={"GEOID": str})
vul = vul.rename(columns = {'vul_score_mulpi':'Multiplicative Vulnerability', 
                            'vul_score_add':'Additive Vulnerability'})
vul = vul[['GEOID', 'Multiplicative Vulnerability', 'Additive Vulnerability']]
vul.head()

In [ ]:
folium_data = pd.merge(exp, sens, on='GEOID',how='outer')
folium_data = pd.merge(folium_data, ac,on='GEOID',how='outer')
folium_data = pd.merge(folium_data, vul,on='GEOID',how='outer')
folium_data = pd.merge(folium_data, basemap,on='GEOID',how='right')
folium_data.head()

In [ ]:
folium_map = gpd.GeoDataFrame(folium_data,crs=4326)

In [ ]:
folium_map.info()

In [ ]:
import jenkspy
exp_data = folium_data['Exposure Score'].fillna(0, inplace=False)
sens_data = folium_data['Sensitivity Score'].fillna(0, inplace=False)
ac_data = folium_data['Adaptive Capacity Score'].fillna(0, inplace=False)
add_vul_data = folium_data['Additive Vulnerability'].fillna(0, inplace=False)
multi_vul_data = folium_data['Multiplicative Vulnerability'].fillna(0, inplace=False)

exp_bins = jenkspy.jenks_breaks(exp_data, nb_class=5)
sens_bins = jenkspy.jenks_breaks(sens_data, nb_class=5)
ac_bins = jenkspy.jenks_breaks(ac_data, nb_class=5)
add_vul_bins = jenkspy.jenks_breaks(add_vul_data, nb_class=5)
multi_vul_bins = jenkspy.jenks_breaks(multi_vul_data, nb_class=5)

In [ ]:
print(exp_bins, '\n', sens_bins, '\n', ac_bins, '\n', add_vul_bins, '\n', multi_vul_bins)

In [ ]:
def add_categorical_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:10px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 12px;
        width: 32px;
        margin-right: 10px;
        margin-left: 10px;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))

    return folium_map

In [ ]:
import folium

m = folium.Map(location=[37, -102], zoom_start=4)

### Exposure map
c = folium.Choropleth(
    geo_data=us_jason,
    name='Exposure',
    data=folium_map,
    columns=['GEOID','Exposure Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Exposure',
    bins=exp_bins,
)
for key in c._children:
    if key.startswith('color_map'):
        del(c._children[key])
c.add_to(m)

### Sensitivity map
d = folium.Choropleth(
    geo_data=us_jason,
    name='Sensitivity',
    data=folium_map,
    columns=['GEOID','Sensitivity Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Sensitivity',
    bins=sens_bins,
)
for key in d._children:
    if key.startswith('color_map'):
        del(d._children[key])
d.add_to(m)

### Adaptive Capacity map
e = folium.Choropleth(
    geo_data=us_jason,
    name='Adaptive Capacity',
    data=folium_map,
    columns=['GEOID','Adaptive Capacity Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Adaptive Capacity',
    bins=ac_bins,
)
for key in e._children:
    if key.startswith('color_map'):
        del(e._children[key])
e.add_to(m)

### Additive Vulnerability map
f = folium.Choropleth(
    geo_data=us_jason,
    name='Additive Vulnerability',
    data=folium_map,
    columns=['GEOID','Additive Vulnerability'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Additive Vulnerability',
    bins=add_vul_bins,
)
for key in f._children:
    if key.startswith('color_map'):
        del(f._children[key])
f.add_to(m)

### Multiplicative Vulnerability map
g = folium.Choropleth(
    geo_data=us_jason,
    name='Multiplicative Vulnerability',
    data=folium_map,
    columns=['GEOID','Multiplicative Vulnerability'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Multiplicative Vulnerability',
    bins=multi_vul_bins,
)
for key in g._children:
    if key.startswith('color_map'):
        del(g._children[key])
g.add_to(m)

# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = folium_map,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                'Additive Vulnerability', 'Multiplicative Vulnerability'],
        aliases=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                 'Additive Vulnerability', 'Multiplicative Vulnerability'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

# Add the customized legend
m = add_categorical_legend(m, 'Exposure',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.0437', '0.0437 - 0.0769', '0.0769 - 0.1130', '0.1130 - 0.1666', '0.1666 - 1.0'])
m = add_categorical_legend(m, 'Sensitivity',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.2272', '0.2272 - 0.4622', '0.4622 - 0.5427', '0.5427 - 0.6505', '0.6505 - 1.0'])
m = add_categorical_legend(m, 'Adaptive Capacity',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.0990', '0.0990 - 0.2500', '0.2500 - 0.3431', '0.3431 - 0.4239', '0.4239 - 1.0'])
m = add_categorical_legend(m, 'Additive Vulnerability',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['-0.6505 - 0.1707', '0.1707 - 0.3860', '0.3860 - 0.5557', '0.5557 - 0.7037', '0.7037 - 1.4384'])
m = add_categorical_legend(m, 'Multiplicative Vulnerability',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['-0.0935 - 0.0167', '0.0167 - 0.0368', '0.0368 - 0.0604', '0.0604 - 0.1000', '0.1000 - 0.4384'])



# Add cross-hatching (crossing lines) to display the Null values.
# nans = exposure_map[exposure_map["Exposure Score"].isnull()]['GEOID'].values
# gdf_nans = exposure_map[exposure_map['GEOID'].isin(nans)]
# sp = StripePattern(angle=45, color='grey', space_color='white')
# sp.add_to(m)
# folium.features.GeoJson(name="NaN values",
#                         data=gdf_nans, 
#                         style_function=lambda x :{'fillPattern': sp},show=False).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save('Folium_all.html')

In [ ]:

m = folium.Map(location=[37, -102], zoom_start=4)

### Exposure map
c = folium.Choropleth(
    geo_data=us_jason,
    name='Exposure',
    data=folium_map,
    columns=['GEOID','Exposure Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Exposure',
    bins=exp_bins,
)
for key in c._children:
    if key.startswith('color_map'):
        del(c._children[key])
c.add_to(m)

# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = folium_map,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                'Additive Vulnerability', 'Multiplicative Vulnerability'],
        aliases=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                 'Additive Vulnerability', 'Multiplicative Vulnerability'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

# Add the customized legend
m = add_categorical_legend(m, 'Exposure',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.0437', '0.0437 - 0.0769', '0.0769 - 0.1130', '0.1130 - 0.1666', '0.1666 - 1.0'])

# Add cross-hatching (crossing lines) to display the Null values.
# nans = exposure_map[exposure_map["Exposure Score"].isnull()]['GEOID'].values
# gdf_nans = exposure_map[exposure_map['GEOID'].isin(nans)]
# sp = StripePattern(angle=45, color='grey', space_color='white')
# sp.add_to(m)
# folium.features.GeoJson(name="NaN values",
#                         data=gdf_nans, 
#                         style_function=lambda x :{'fillPattern': sp},show=False).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save('Folium_exp.html')

In [ ]:
m = folium.Map(location=[37, -102], zoom_start=4)

### Sensitivity map
d = folium.Choropleth(
    geo_data=us_jason,
    name='Sensitivity',
    data=folium_map,
    columns=['GEOID','Sensitivity Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Sensitivity',
    bins=sens_bins,
)
for key in d._children:
    if key.startswith('color_map'):
        del(d._children[key])
d.add_to(m)

### Adaptive Capacity map
e = folium.Choropleth(
    geo_data=us_jason,
    name='Adaptive Capacity',
    data=folium_map,
    columns=['GEOID','Adaptive Capacity Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Adaptive Capacity',
    bins=ac_bins,
)
for key in e._children:
    if key.startswith('color_map'):
        del(e._children[key])
e.add_to(m)

### Additive Vulnerability map
f = folium.Choropleth(
    geo_data=us_jason,
    name='Additive Vulnerability',
    data=folium_map,
    columns=['GEOID','Additive Vulnerability'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Additive Vulnerability',
    bins=add_vul_bins,
)
for key in f._children:
    if key.startswith('color_map'):
        del(f._children[key])
f.add_to(m)

### Multiplicative Vulnerability map
g = folium.Choropleth(
    geo_data=us_jason,
    name='Multiplicative Vulnerability',
    data=folium_map,
    columns=['GEOID','Multiplicative Vulnerability'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Multiplicative Vulnerability',
    bins=multi_vul_bins,
)
for key in g._children:
    if key.startswith('color_map'):
        del(g._children[key])
g.add_to(m)

# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = folium_map,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                'Additive Vulnerability', 'Multiplicative Vulnerability'],
        aliases=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                 'Additive Vulnerability', 'Multiplicative Vulnerability'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

# Add the customized legend
m = add_categorical_legend(m, 'Exposure',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.0437', '0.0437 - 0.0769', '0.0769 - 0.1130', '0.1130 - 0.1666', '0.1666 - 1.0'])
m = add_categorical_legend(m, 'Sensitivity',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.2272', '0.2272 - 0.4622', '0.4622 - 0.5427', '0.5427 - 0.6505', '0.6505 - 1.0'])
m = add_categorical_legend(m, 'Adaptive Capacity',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.0990', '0.0990 - 0.2500', '0.2500 - 0.3431', '0.3431 - 0.4239', '0.4239 - 1.0'])
m = add_categorical_legend(m, 'Additive Vulnerability',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['-0.6505 - 0.1707', '0.1707 - 0.3860', '0.3860 - 0.5557', '0.5557 - 0.7037', '0.7037 - 1.4384'])
m = add_categorical_legend(m, 'Multiplicative Vulnerability',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['-0.0935 - 0.0167', '0.0167 - 0.0368', '0.0368 - 0.0604', '0.0604 - 0.1000', '0.1000 - 0.4384'])



# Add cross-hatching (crossing lines) to display the Null values.
# nans = exposure_map[exposure_map["Exposure Score"].isnull()]['GEOID'].values
# gdf_nans = exposure_map[exposure_map['GEOID'].isin(nans)]
# sp = StripePattern(angle=45, color='grey', space_color='white')
# sp.add_to(m)
# folium.features.GeoJson(name="NaN values",
#                         data=gdf_nans, 
#                         style_function=lambda x :{'fillPattern': sp},show=False).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save('Folium_all.html')

# Exposure

In [ ]:
exposure = pd.read_csv('Exposure_0412.csv', dtype = {'State':str,"GEOID":str,"GEO_ID":str})
# exposure = exposure.rename(columns = {'min-max':'Exposure Score'})
exposure.head()

In [ ]:
exposure_map = pd.merge(exposure,basemap,on='GEOID',how='right')

In [ ]:
exposure_map.info()

In [ ]:
exposure_map = gpd.GeoDataFrame(exposure_map,crs=4326)

In [ ]:
exposure_map.crs

In [ ]:
exp_data = exposure_map['Exposure Score'].fillna(0, inplace=False)

In [ ]:
pip install jenkspy

In [ ]:
import jenkspy

breaks = jenkspy.jenks_breaks(exp_data, nb_class=5)
print(breaks)

In [ ]:
def add_categorical_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 32px;
        margin-right: 10px;
        margin-left: 10px;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))

    return folium_map

In [ ]:
import folium

m = folium.Map(location=[37, -102], zoom_start=4)

c = folium.Choropleth(
    geo_data=us_jason,
    name='Exposure',
    data=exposure_map,
    columns=['GEOID','Exposure Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Exposure',
    bins=breaks,
)

for key in c._children:
    if key.startswith('color_map'):
        del(c._children[key])

c.add_to(m)        

# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = exposure_map,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Exposure Score'],
        aliases=['Exposure Score'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

# Add the customized legend
m = add_categorical_legend(m, 'Exposure',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0-0.0437', '0.0437-0.0769', '0.0769-0.1130', '0.1130-0.1666', '0.1666-1.0'])

# Add cross-hatching (crossing lines) to display the Null values.
nans = exposure_map[exposure_map["Exposure Score"].isnull()]['GEOID'].values
gdf_nans = exposure_map[exposure_map['GEOID'].isin(nans)]
sp = StripePattern(angle=45, color='grey', space_color='white')
sp.add_to(m)
folium.features.GeoJson(name="NaN values",
                        data=gdf_nans, 
                        style_function=lambda x :{'fillPattern': sp},show=False).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save('exposure_OrRd.html')

# Sensitivity

In [ ]:
sensitivity = pd.read_csv("Sensitivity-0409.csv", dtype={"Geocode": str})

In [ ]:
sensitivity.head()

In [ ]:
sensitivity_map = pd.merge(sensitivity,basemap,left_on='Geocode',right_on='GEOID',how='right').drop(['Geocode'],axis=1)

In [ ]:
sensitivity_map.head()

In [ ]:
sensitivity_map = gpd.GeoDataFrame(sensitivity_map,crs = 4326)

In [ ]:
sensitivity_map.crs

In [ ]:
import jenkspy

sen_data = sensitivity_map['min_max_sens'].fillna(0, inplace=False)

breaks = jenkspy.jenks_breaks(sen_data, nb_class=5)
print(breaks)

In [ ]:
def add_categorical_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 32px;
        margin-right: 10px;
        margin-left: 10px;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))

    return folium_map

In [ ]:
import folium

m = folium.Map(location=[37, -102], zoom_start=4)

c = folium.Choropleth(
    geo_data=us_jason,
    name='choropleth',
    data=sensitivity_map,
    columns=['GEOID','min_max_sens'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = '////',
    legend_name='Sensitivity',
    bins=breaks,
)

for key in c._children:
    if key.startswith('color_map'):
        del(c._children[key])

c.add_to(m)        

m = add_categorical_legend(m, 'Sensitivity',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0-0.0437', '0.0437-0.0769', '0.0769-0.1130', '0.1130-0.1666', '0.1666-1.0'])

folium.LayerControl().add_to(m)

m.save('sensitivity_OrRd.html')

# Adaptive Capacity

In [ ]:
adaptive_capacity = pd.read_csv('all_ac_data_0516.csv',dtype={"GEO_ID":str})
adaptive_capacity.head()

In [ ]:
adaptive_capacity.describe()

In [ ]:
adaptive_capacity_map = pd.merge(adaptive_capacity,basemap,left_on='GEO_ID',right_on='GEOID',how='right').drop(['GEO_ID'],axis=1)

In [ ]:
adaptive_capacity_map.info()

In [ ]:
adaptive_capacity_map = gpd.GeoDataFrame(adaptive_capacity_map,crs = albo.proj4_init)

In [ ]:
adaptive_capacity_map.crs

In [ ]:
adaptive_capacity_map.plot(column='AC_score')

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

adaptive_capacity_map.plot(
    column='AC_score',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Adaptive Capacity',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # 图例标题字体大小
                 'fontsize': 11, # 图例非标题外字体大小
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US - Adaptive Capacity',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Adaptive Capacity_USA.png', facecolor='w', dpi=500)

# Vulnerabilty Score

In [ ]:
e = exposure_map[['GEOID','min-max']]
s = sensitivity_map[['GEOID','min_max_sens']]
ac = adaptive_capacity_map[['GEOID','AC_score']]

In [ ]:
vulnerablity = pd.merge(e, s, on="GEOID")
vulnerablity = pd.merge(vulnerablity, ac, on="GEOID")
vulnerablity.head()

In [ ]:
vulnerablity.info()

In [ ]:
vulnerablity.fillna(0, inplace=True)

In [ ]:
vulnerablity['vul_score_mulpi'] = vulnerablity.apply(lambda x: x["min-max"]*(x["min_max_sens"]-x["AC_score"]), axis=1)

In [ ]:
mc.NaturalBreaks(vulnerablity['vul_score_add'])

In [ ]:
mc.NaturalBreaks(vulnerablity_map_pos['vul_score_mulpi'])

In [ ]:
mc.NaturalBreaks(vulnerablity_map_neg['vul_score_mulpi'])

In [ ]:
print(vulnerablity['vul_score_mulpi'].mean(),vulnerablity['vul_score_mulpi'].std())

In [ ]:
vulnerablity['vul_mulpi_1/3'] = vulnerablity.apply(lambda x: 1/3*x["min-max"]*(1/3*x["min_max_sens"]-1/3*x["AC_score"]), axis=1)

In [ ]:
vulnerablity['vul_score_add'] = vulnerablity.apply(lambda x: x["min-max"]+x["min_max_sens"]-x["AC_score"], axis=1)

In [ ]:
print(vulnerablity['vul_score_add'].mean(),vulnerablity['vul_score_add'].std())

In [ ]:
print(vulnerablity['vul_score_add'].max(),vulnerablity['vul_score_add'].min())

In [ ]:
vulnerablity['vul_add_1/3'] = vulnerablity.apply(lambda x: 1/3*x["min-max"]+1/3*x["min_max_sens"]-1/3*x["AC_score"], axis=1)

In [ ]:
vulnerablity.head()

In [ ]:
vulnerablity.to_csv('Vulnerability Score.csv')

In [ ]:
vulnerablity['vul_score_mulpi'].min()

In [ ]:
vulnerablity_map = pd.merge(vulnerablity,basemap,on='GEOID')

In [ ]:
vulnerablity_map.info()

In [ ]:
vulnerablity_map = gpd.GeoDataFrame(vulnerablity_map,crs = albo.proj4_init)

In [ ]:
vulnerablity_map.crs

In [ ]:
vulnerablity_map_df = pd.DataFrame(vulnerablity_map)
vulnerablity_map_df.describe()

In [ ]:
vulnerablity_map_pos = vulnerablity_map[vulnerablity_map['vul_score_mulpi']>=0]
vulnerablity_map_neg = vulnerablity_map[vulnerablity_map['vul_score_mulpi']<0]

In [ ]:
print(vulnerablity_map_pos['vul_score_mulpi'].mean(),vulnerablity_map_pos['vul_score_mulpi'].std())
print(vulnerablity_map_neg['vul_score_mulpi'].mean(),vulnerablity_map_neg['vul_score_mulpi'].std())

In [ ]:
print(len(vulnerablity_map_pos), len(vulnerablity_map_neg))

In [ ]:
plt.figure(figsize=(10, 10), dpi=100)
code = np.arange(0,len(vulnerablity_map))
plt.scatter(,vulnerablity_map['vul_score_add'])
plt.show()

In [ ]:
vulnerablity_map_neg

In [ ]:
vulnerablity_map_neg.STATEFP.unique()

In [ ]:
vulnerablity_map_pos.info()

In [ ]:
vulnerablity_map_pos_41 = vulnerablity_map_pos[vulnerablity_map_pos['STATEFP']=='41']
vulnerablity_map_neg_41 = vulnerablity_map_neg[vulnerablity_map_neg['STATEFP']=='41']

In [ ]:
import numpy as np
import mapclassify as mc

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.cmocean.sequential import Amp_20
from palettable.cmocean.sequential import Ice_20

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

cmap1 = Amp_20.mpl_colormap
cmap2 = Ice_20.mpl_colormap

vulnerablity_map_neg_49.plot(
    column='vul_score_mulpi',
    ax=ax,
    cmap=cmap1,
    legend=False,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'lower left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # 图例标题字体大小
                 'fontsize': 11, # 图例非标题外字体大小
                 'shadow': True
                },               
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.1
)

vulnerablity_map_pos_49.plot(
    column='vul_score_mulpi',
    ax=ax,
    cmap=cmap,
    legend=False,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # 图例标题字体大小
                 'fontsize': 11, # 图例非标题外字体大小
                 'shadow': True
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.1
)



state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/49.png', facecolor='w', dpi=500)

In [ ]:
print(len(vulnerablity_map_pos),len(vulnerablity_map_neg))

In [ ]:
print(len(vulnerablity_map))

In [ ]:
import numpy as np
import mapclassify as mc

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.cmocean.sequential import Amp_20
from palettable.cmocean.sequential import Ice_20

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

cmap1 = Amp_20.mpl_colormap
cmap2 = Ice_20.mpl_colormap

vulnerablity_map_neg.plot(
    column='vul_score_mulpi',
    ax=ax,
    cmap=cmap1,
    legend=False,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'lower left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # 图例标题字体大小
                 'fontsize': 11, # 图例非标题外字体大小
                 'shadow': True
                },               
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.1
)

vulnerablity_map_pos.plot(
    column='vul_score_mulpi',
    ax=ax,
    cmap=cmap,
    legend=False,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # 图例标题字体大小
                 'fontsize': 11, # 图例非标题外字体大小
                 'shadow': True
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.1
)



state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Vul_score_mulpi_0726.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np
import mapclassify as mc

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-125, -117, 42, 50], crs=ccrs.PlateCarree())

from palettable.cmocean.sequential import Amp_20
from palettable.cmocean.sequential import Ice_20

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

cmap1 = Amp_20.mpl_colormap
cmap2 = Ice_20.mpl_colormap

vulnerablity_map_neg.plot(
    column='vul_score_mulpi',
    ax=ax,
    cmap=cmap1,
    legend=False,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'lower left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # 图例标题字体大小
                 'fontsize': 11, # 图例非标题外字体大小
                 'shadow': True
                },               
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.1
)

vulnerablity_map_pos.plot(
    column='vul_score_mulpi',
    ax=ax,
    cmap=cmap,
    legend=False,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # 图例标题字体大小
                 'fontsize': 11, # 图例非标题外字体大小
                 'shadow': True
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.1
)



state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/WS.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(14, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

vulnerablity_map_pos.plot(
    column='vul_score_mulpi',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'lower left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # 图例标题字体大小
                 'fontsize': 11, # 图例非标题外字体大小
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/lengend1.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-115, -90, 35, 50], crs=ccrs.PlateCarree())

from palettable.cmocean.sequential import Amp_20

cmap = Amp_20.mpl_colormap

vulnerablity_map_neg.plot(
    column='vul_score_mulpi',
    ax=ax,
    cmap=cmap,
    legend=False,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # 图例标题字体大小
                 'fontsize': 11, # 图例非标题外字体大小
                 'shadow': True
                },               
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='black',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/midareas.png', facecolor='w', dpi=500)

In [ ]:
WS = state_boundary[state_boundary['STATEFP']=='53']
WS.crs

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(12, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)

#设置图框线粗细
bwith = 1.5 #边框宽度设置为2
TK = plt.gca()#获取边框
TK.spines['bottom'].set_linewidth(bwith)
TK.spines['left'].set_linewidth(bwith)
TK.spines['top'].set_linewidth(bwith)
TK.spines['right'].set_linewidth(bwith)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

vulnerablity_map.plot(
    column='vul_mulpi_1/3',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Vulnerablity Score',
                 'shadow': True
                },
#     missing_kwds={
#                 "color": "lightgrey",
#                 "edgecolor": "grey",
#                 "hatch": "////",
#                 "label": "Nan"
#                 },
    edgecolor=np.array([0., 0., 0., 0.]),
    linewidth=0.1
)

ax.axis('off')
ax.set_title('USA Transportation Energy Vulnerability', font='Times New Roman', fontsize=20,pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Vul_score_multiple_1_3.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(
    subplot_kw={'projection': albo}, figsize=(10, 10),dpi=300
)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)
ax.set_extent([-129, -73, 20, 50], crs=ccrs.PlateCarree())

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

vulnerablity_map.plot(
    column='vul_score_add',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
    scheme='NaturalBreaks',
#     k=7,
    legend_kwds={'loc': 'upper left',
                 'title': 'Vulnerability Score',
                 "fmt": "{:.3f}",
                 'title_fontsize': 14, # 图例标题字体大小
                 'fontsize': 11, # 图例非标题外字体大小
                 'shadow': True
                },               
    missing_kwds={
                "color": "lightgrey",
                "edgecolor": "grey",
                "hatch": "////",
                "label": "Nan"
                },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2
)
state_boundary.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/Vul_score_add.png', facecolor='w', dpi=500)

# Weight Testing

In [ ]:
vul_weidghted = pd.merge(e, s, on="GEOID")
vul_weidghted = pd.merge(vul_weidghted, ac, on="GEOID")
vul_weidghted.head()

In [ ]:
vul_weidghted.fillna(0, inplace=True)

In [ ]:
# min-max: exposure, min_max_sens: sensitivity; AC_score: adaptive capacity
# Suppose the weight are 0.6 , 0.2, 0.2
vul_weidghted['vul_mulpi_622'] = vul_weidghted.apply(lambda x: 1.8*x["min-max"]*(0.6*x["min_max_sens"]-0.6*x["AC_score"]), axis=1)
vul_weidghted['vul_add_622'] = vul_weidghted.apply(lambda x: 1.8*x["min-max"]+0.6*x["min_max_sens"]-0.6*x["AC_score"], axis=1)

In [ ]:
vul_weidghted['vul_mulpi_622'].corr(vulnerablity['vul_mulpi_1/3'])

In [ ]:
vul_weidghted['vul_mulpi_622'].corr(vulnerablity['vul_score_mulpi'])

In [ ]:
vul_weidghted['vul_add_622'].corr(vulnerablity['vul_add_1/3'])

In [ ]:
vul_weidghted['vul_add_622'].corr(vulnerablity['vul_score_add'])

In [ ]:
# min-max: exposure, min_max_sens: sensitivity; AC_score: adaptive capacity
# Suppose the weight are 0.9 , 0.05, 0.05
vul_weidghted['vul_mulpi_955'] = vul_weidghted.apply(lambda x: 2.7*x["min-max"]*(0.15*x["min_max_sens"]-0.15*x["AC_score"]), axis=1)

In [ ]:
vul_weidghted['vul_add_955'] = vul_weidghted.apply(lambda x: 2.7*x["min-max"]+0.15*x["min_max_sens"]-0.15*x["AC_score"], axis=1)

In [ ]:
vul_weidghted['vul_mulpi_955'].corr(vulnerablity['vul_mulpi_1/3'])

In [ ]:
vul_weidghted['vul_mulpi_955'].corr(vulnerablity['vul_score_mulpi'])

In [ ]:
vul_weidghted['vul_add_955'].corr(vulnerablity['vul_add_1/3'])

In [ ]:
vul_weidghted['vul_add_955'].corr(vulnerablity['vul_score_add'])

In [ ]:
# Suppose the weight are 0.45 , 0.45, 0.1
vul_weidghted['vul_mulpi_45451'] = vul_weidghted.apply(lambda x: 0.45*x["min-max"]*(0.45*x["min_max_sens"]-0.1*x["AC_score"]), axis=1)

In [ ]:
vul_weidghted['vul_add_45451'] = vul_weidghted.apply(lambda x: 0.45*x["min-max"]+0.45*x["min_max_sens"]-0.1*x["AC_score"], axis=1)

In [ ]:
vul_weidghted['vul_mulpi_45451'].corr(vulnerablity['vul_mulpi_1/3'])

In [ ]:
vul_weidghted['vul_add_45451'].corr(vulnerablity['vul_add_1/3'])

In [ ]:
# Suppose the weight are 0.2 , 0.6, 0.2
vul_weidghted['vul_mulpi_262'] = vul_weidghted.apply(lambda x: 0.6*x["min-max"]*(1.8*x["min_max_sens"]-0.6*x["AC_score"]), axis=1)

In [ ]:
vul_weidghted['vul_mulpi_595'] = vul_weidghted.apply(lambda x: 0.05*x["min-max"]*(0.9*x["min_max_sens"]-0.05*x["AC_score"]), axis=1)

In [ ]:
vul_weidghted['vul_add_595'] = vul_weidghted.apply(lambda x: 0.05*x["min-max"]+0.9*x["min_max_sens"]-0.05*x["AC_score"], axis=1)

In [ ]:
vul_weidghted['vul_mulpi_262'].corr(vulnerablity['vul_mulpi_1/3'])

In [ ]:
vul_weidghted['vul_add_weighted_6'].corr(vulnerablity['vul_add_1/3'])

In [ ]:
vul_weidghted['vul_add_262'] = vul_weidghted.apply(lambda x: 0.2*x["min-max"]+0.6*x["min_max_sens"]-0.2*x["AC_score"], axis=1)

In [ ]:
vul_weidghted['vul_mulpi_weighted_3'].corr(vulnerablity['vul_mulpi_1/3'])

In [ ]:
vul_weidghted['vul_add_weighted_3'].corr(vulnerablity['vul_add_1/3'])

In [ ]:
# Suppose the weight are 0.2 , 0.2, 0.6
vul_weidghted['vul_mulpi_226'] = vul_weidghted.apply(lambda x: 0.2*x["min-max"]*(0.2*x["min_max_sens"]-0.6*x["AC_score"]), axis=1)

In [ ]:
vul_weidghted['vul_add_226'] = vul_weidghted.apply(lambda x: 0.2*x["min-max"]+0.2*x["min_max_sens"]-0.6*x["AC_score"], axis=1)

In [ ]:
# Suppose the weight are 0.2 , 0.2, 0.6
vul_weidghted['vul_mulpi_559'] = vul_weidghted.apply(lambda x: 0.05*x["min-max"]*(0.05*x["min_max_sens"]-0.9*x["AC_score"]), axis=1)

In [ ]:
vul_weidghted['vul_add__559'] = vul_weidghted.apply(lambda x: 0.05*x["min-max"]+0.05*x["min_max_sens"]-0.9*x["AC_score"], axis=1)

In [ ]:
vul_weidghted['vul_mulpi_weighted_7'].corr(vulnerablity['vul_mulpi_1/3'])

In [ ]:
vul_weidghted['vul_add_weighted_7'].corr(vulnerablity['vul_add_1/3'])

In [ ]:
vul_weidghted['vul_mulpi_weighted_4'].corr(vulnerablity['vul_mulpi_1/3'])

In [ ]:
vul_weidghted['vul_add_weighted_4'].corr(vulnerablity['vul_add_1/3'])

In [ ]:
vul_weidghted

In [ ]:
vul_weidghted.to_excel('vul_weidght_test.xlsx')

# Switch CV to EV IL

In [ ]:
cv_ev_ex = pd.read_excel('IL CV-EV/IL_CV2EV_base.xlsx',dtype={'CountyFIPs':str})
cv_ev_ex.head()

In [ ]:
ex_IL = cv_ev_ex[['CountyFIPs','County', '2020_CV Costexpo_nor', '2030_CV Costexpo_nor',
               '2040_CV Costexpo_nor','2050_CV Costexpo_nor','2020_EV Costexpo_nor','2030_EV Costexpo_nor',
               '2040_EV Costexpo_nor','2050_EV Costexpo_nor']]
ex_IL.head()

In [ ]:
s_IL = sensitivity_map[['GEOID','min_max_sens']]
ac_IL = adaptive_capacity_map[['GEOID','AC_score']]

In [ ]:
pop = pd.read_csv('IL CV-EV/latch_2017.csv',dtype={'geocode':str})
for i in range(len(pop)):
    pop.loc[i,'geocode'] = pop.loc[i,'geocode'].zfill(11)
pop.head()

In [ ]:
col_name=pop.columns.tolist()
col_name.insert(1,'StateFIPs')
col_name.insert(2,'CountyFIPs')
col_name.insert(3,'CountyPOP')
pop=pop.reindex(columns=col_name)
for i in range(len(pop)):
    pop.loc[i,'StateFIPs'] = str(pop.loc[i,'geocode'])[:2]
    pop.loc[i,'CountyFIPs'] = str(pop.loc[i,'geocode'])[:5]

In [ ]:
pop_IL = pop[(pop.StateFIPs == "17")]
pop_IL = pop_IL.reset_index()

In [ ]:
pop_county = pop_IL.groupby(['CountyFIPs'])[['tot_pop']].sum()
pop_county = pop_county.reset_index()
pop_county.head()

In [ ]:
for i in range(len(pop_IL)):
    for j in range(len(pop_county)):
        if pop_IL.loc[i,'CountyFIPs'] == pop_county.loc[j,'CountyFIPs']:
            pop_IL.loc[i,'CountyPOP'] = pop_county.loc[j,'tot_pop']

In [ ]:
pop_IL

In [ ]:
s_IL = pd.merge(s_IL,pop_IL,left_on='GEOID',right_on='geocode',how="left")
ac_IL = pd.merge(ac_IL,pop_IL,left_on='GEOID',right_on='geocode',how="left")

In [ ]:
s_IL = s_IL[(s_IL.StateFIPs == "17")]
ac_IL = ac_IL[(ac_IL.StateFIPs == "17")]

In [ ]:
s_IL['sens-new'] = s_IL.apply(lambda x: x['min_max_sens'] * (x['tot_pop']/x['CountyPOP']),axis=1)
ac_IL['ac-new'] = ac_IL.apply(lambda x: x['AC_score'] * (x['tot_pop']/x['CountyPOP']),axis=1)

In [ ]:
s_county = s_IL.groupby(['CountyFIPs'])[['sens-new']].sum()
s_county.head()

In [ ]:
ac_county = ac_IL.groupby(['CountyFIPs'])[['ac-new']].sum()
ac_county.head()

In [ ]:
s_county = s_county.reset_index()
ac_county = ac_county.reset_index()

In [ ]:
minsens = s_county['sens-new'].min()
maxsens = s_county['sens-new'].max()
minac = ac_county['ac-new'].min()
maxac = ac_county['ac-new'].max()
s_county['sens-new'] = s_county.apply(lambda x: (x['sens-new'] - minsens)/(maxsens-minsens),axis=1)
ac_county['ac-new'] = ac_county.apply(lambda x: (x['ac-new'] - minac)/(maxac-minac),axis=1)

In [ ]:
ac_county.head()

In [ ]:
vul_score = pd.merge(ex_IL,s_county,on='CountyFIPs')
vul_score = pd.merge(vul_score,ac_county,on='CountyFIPs')
vul_score.head()

In [ ]:
vul_score.describe()

In [ ]:
vul_score.fillna(0, inplace=True)

In [ ]:
vul_score['2020CV'] = vul_score.apply(lambda x: x["2020_CV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2030CV'] = vul_score.apply(lambda x: x["2030_CV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2040CV'] = vul_score.apply(lambda x: x["2040_CV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2050CV'] = vul_score.apply(lambda x: x["2050_CV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2020EV'] = vul_score.apply(lambda x: x["2020_EV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2030EV'] = vul_score.apply(lambda x: x["2030_EV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2040EV'] = vul_score.apply(lambda x: x["2040_EV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)
vul_score['2050EV'] = vul_score.apply(lambda x: x["2050_EV Costexpo_nor"]*(x["sens-new"]-x["ac-new"]), axis=1)

In [ ]:
vul_score.head()

In [ ]:
vul_score.to_excel('IL CV-EV/vul_score CV-EV_base.xlsx')

In [ ]:
vul_IL = pd.read_excel('IL CV-EV/vul_score CV-EV_base.xlsx',dtype={'CountyFIPs':str})
vul_IL.info()

In [ ]:
IL_map = gpd.read_file('IL_county/IL_BNDY_County_Py.shp',dtype={'CO_FIPS':str})
IL_map.crs

In [ ]:
IL_vul_map = pd.merge(vul_IL, IL_map, left_on = 'County', right_on = 'COUNTY_NAM')

In [ ]:
IL_vul_map = gpd.GeoDataFrame(IL_vul_map,crs = 4269)

In [ ]:
IL_vul_map['2020CV_new'] = IL_vul_map.apply(lambda x: abs(x['2020CV']), axis=1)

In [ ]:
IL_vul_map['2020EV_new'] = IL_vul_map.apply(lambda x: abs(x['2020EV']), axis=1)

In [ ]:
IL_vul_map['dif_20'] = IL_vul_map.apply(lambda x: x['2020CV_new'] - x['2020EV_'], axis=1)

In [ ]:
print(IL_vul_map['2020CV_new'].max(),IL_vul_map['2020CV_new'].min())

In [ ]:
print(IL_vul_map['2020EV_new'].max(),IL_vul_map['2020EV_new'].min())

In [ ]:
IL_vul_map['dif_50'] = IL_vul_map.apply(lambda x: x['2050CV'] - x['2050EV'], axis=1)  

In [ ]:
IL_vul_map['2050CV_new'] = IL_vul_map.apply(lambda x: abs(x['2050CV']), axis=1)

In [ ]:
IL_vul_map['2050EV_new'] = IL_vul_map.apply(lambda x: abs(x['2050EV']), axis=1)

In [ ]:
print(IL_vul_map['2050CV_new'].max(),IL_vul_map['2050CV_new'].min())

In [ ]:
print(IL_vul_map['2050EV_new'].max(),IL_vul_map['2050EV_new'].min())

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(10, 10),dpi=300)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

IL_vul_map.plot(
    column='2050CV_new',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
#    scheme='NaturalBreaks',
#     k=7,
#     legend_kwds={'title': 'Vulnerability Score',
#                  "fmt": "{:.3f}",
#                  'title_fontsize': 14, # 图例标题字体大小
#                  'fontsize': 11, # 图例非标题外字体大小
# #                  'shadow': True
#                 },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2,
    vmin = 0,
    vmax = 0.7442712535493357
)
IL_map.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
#ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/IL_2050CV.png', facecolor='w', dpi=500)

In [ ]:
import numpy as np

fig, ax = plt.subplots(figsize=(10, 10),dpi=300)
font = {'family': 'serif',
        'serif': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)

from palettable.matplotlib import Viridis_7

cmap = Viridis_7.mpl_colormap

IL_vul_map.plot(
    column='2050EV_new',
    ax=ax,
    cmap=cmap,
    legend=True,
#     legend_kwds={'label': 'Exposure'},
#    scheme='NaturalBreaks',
#     k=7,
#     legend_kwds={'loc': 'lower left',
#                  'title': 'Vulnerability Score',
#                  "fmt": "{:.3f}",
#                  'title_fontsize': 14, # 图例标题字体大小
#                  'fontsize': 11, # 图例非标题外字体大小
#                  'shadow': True
#                 },
#     prop={'family': 'Times New Roman', 'size': 16},
    edgecolor=np.array([0., 0., 0., 0.]), #'white',
    linewidth=0.2,
    vmin = 0,
    vmax = 0.7442712535493357
)
IL_map.boundary.plot(ax=ax,edgecolor='white',linewidth=0.5)

ax.axis('on')
#ax.set_title('Transportation Energy Vulnerability in the US',fontdict={'weight':'bold','size': 20},pad=20)
plt.tight_layout(pad=4.5)


ax.set_aspect('equal','box')
# fig.tight_layout()

fig.savefig('Figures/IL_2050EV.png', facecolor='w', dpi=500)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.violinplot(
               y = "2020CV", 
               data = vul_score
              )
plt.show()

In [ ]:
sns.violinplot(
               y = "2030CV", 
               data = vul_score
              )
plt.show()

In [ ]:
sns.violinplot(
               y = "2020EV", 
               data = vul_score
              )
plt.show()

In [ ]:
sns.violinplot(
               y = "2030EV", 
               data = vul_score
              )
plt.show()

In [ ]:
sns.violinplot(
               y = "2040EV", 
               data = vul_score
              )
plt.show()

In [ ]:
sns.violinplot(
               y = "2050EV", 
               data = vul_score
              )
plt.show()

In [ ]:
PADD1A = ['09','23','25','33','44','50']
PADD1B = ['10','11','24','34','36','42']
PADD1C = ['12','13','37','45','51','54']
PADD2 = ['17','18','19','20','21','26','27','29','31','38','39','40','46','47','55']
PADD3 = ['01','05','22','28','35','48']
PADD4 = ['08','16','30','49','56']
PADD5 = ['02','04','06','15','32','41','53']

In [ ]:
state_boundary.plot()

In [ ]:
state_boundary.boundary.plot()